In [1]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 1.1 MB/s eta 0:00:00m eta 0:00:010:00:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.6/813.6 kB 1.6 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 1.0 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 1.6 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.0/39.0 MB 1.4 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 1.3 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.7/238.7 kB 627.9 kB/s eta 0:00:00m eta 0:00:010:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 1.1 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 1.2 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 5

In [1]:
!pip install streamlit-folium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.4/423.4 kB 1.6 MB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [1]:
import streamlit as st
import pandas as pd
import pickle

### Create dropdown list for street name

In [5]:
df = pd.read_csv('NJOP_coordinate.csv')
df.head()
df_njop = pd.read_csv('NJOP2021.csv')
df_njop = df_njop.drop('Unnamed: 0', axis=1)
cols = ['provinsi', 'kota', 'kecamatan', 'kelurahan']
for col in cols:
    df_njop[col] = df_njop[col].apply(lambda x: str(x).split(' - ')[1])
df_njop = df_njop[['kota', 'kecamatan', 'kelurahan', 'nama_jalan']]
df_njop = df_njop.merge(df, on=['kelurahan', 'nama_jalan'], how='inner').drop_duplicates()
df_njop = df_njop[ df_njop['kota'] != 'KEPULAUAN SERIBU' ]
df_njop = df_njop.drop('ave_njb', axis=1)
remove_spaces = ['P U L O', 'C I K O K O', 'P A N C O R A N', 'R A G U N A N', 'S E L O N G', 'S E N A Y A N', 'K A R E T', 'G U N T U R', 'KALI BATA', 'B A N G K A', 'U L U J A M I', 'C I P U L I R', 'PAL MERIAM', 'KALI ANYAR', 'RAWA JATI', 'SETIA BUDI']
df_njop['kelurahan'] = df_njop['kelurahan'].apply(lambda x: x.replace(' ', '') if x in remove_spaces else x)
df_njop['kelurahan'] = df_njop['kelurahan'].str.replace('KEBAYORAN LAMA UTR', 'KEBAYORAN LAMA UTARA').str.replace('KEBAYORAN LAMA SLT', 'KEBAYORAN LAMA SELATAN').str.replace('BALIMESTER', 'BALI MESTER')
remove_spaces = ['C I L A N D A K', 'J A G A K A R S A', 'P A N C O R A N', 'T E B E T', 'SETIA BUDI']
df_njop['kecamatan'] = df_njop['kecamatan'].apply(lambda x: x.replace(' ', '') if x in remove_spaces else x)
df_njop.to_csv('kota_kecamatan_kelurahan_jalan_latlong.csv', index=False)
del df, df_njop

In [6]:
df = pd.read_csv('kota_kecamatan_kelurahan_jalan_latlong.csv')

,kota,kecamatan,kelurahan,nama_jalan,latitude,longitude
0,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG JATI INDAH,-6.259963,106.773803
1,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG MASRIKI,-6.262643,106.774113
2,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,H SAIMAN,-6.261486,106.774134
3,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,JATI INDAH,-6.259963,106.773803
4,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,JL BANK EXIM,-6.263339,106.774593


In [12]:
df_facility = pd.read_csv('gmaps_scrape_data_cleaned.csv')
df_facility = df_facility.drop('Unnamed: 0', axis=1)
df_facility['latitude'] = df_facility['coordinate'].apply(lambda x: x.split(',')[0])
df_facility['longitude'] = df_facility['coordinate'].apply(lambda x: x.split(',')[1])
df_facility['category'] = None
df_facility['category'][:84] = 'mall'
df_facility['category'][84:135] = 'train_station'
df_facility['category'][135:349] = 'bus_station'
df_facility['category'][349:] = 'hospital'
df_facility.to_csv('facility_latlong.csv', index=False)
del df_facility

/tmp/ipykernel_4746/615041234.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_facility['category'][:84] = 'mall'
/tmp/ipykernel_4746/615041234.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_facility['category'][84:135] = 'train_station'
/tmp/ipykernel_4746/615041234.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_facility['category'][135:349] = 'bus_station'
/tmp/ipykernel_4746/615041234.py:9: SettingWithCopyWarni

In [10]:
st.selectbox('Pilih kota:', tuple(sorted(df['kota'].unique())))

('JAKARTA BARAT',
 'JAKARTA PUSAT',
 'JAKARTA SELATAN',
 'JAKARTA TIMUR',
 'JAKARTA UTARA')